# Node2vec Implementation

This algorithm contains two steps:
1. Generate sequences from graph by using random walk
2. Use word2vec model to train embedding matrix from sequences

In [4]:
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import *
%matplotlib inline

In [11]:
# prepare the graph
row  = np.array([0, 3, 1, 0])
col  = np.array([0, 3, 1, 2])
data = np.array([4, 5, 7, 9])
A = coo_matrix((data, (row, col)), shape=(4, 4)).tocsr() # store it as row matrix for better performance

## Random Walk by using search bias alpha
Let $t$ be the previous node, $v$ be the current node, so the $\pi_{vx}$ for each $x$ node in the graph is defined as below:
$$\pi_{vx} = \alpha_{pq}(t,x)w_{vx}$$
$$\alpha_{pq}(t,x) = \left\{\begin{array}[ll]
                \frac{1}{p} & \mbox{if}~d_{tx} = 0\\
                1 & \mbox{if}~d_{tx} = 1\\
                \frac{1}{q} & \mbox{if}~d_{tx} = 2
           \end{array}\right.$$

In [42]:
def alpha(t, x, p, q, data_graph):
    if t == x:
        return 1.0/p
    elif data_graph[t, x] != 0:
        return 1
    else:
        return 1.0/q

def random_walk(start, p, q, sequence_length, data_graph):
    sequence = []
    for step in xrange(sequence_length):
        if step == 0:
            sequence.append(start)
        elif step == 1:
            if data_graph
            current = data_graph[sequence[step-1],:].toarray().flatten()
            if sum(current) != 0:
                sequence.append(np.random.choice(range(data_graph.shape[1]), p=current/np.linalg.norm(current, ord=1)))
            else:
                sequence.append(sequence[step-1])
        else:
            current = data_graph[sequence[step-1],:].toarray().flatten()
            if sum(current) != 0:
                past = sequence[step-2]
                pi_list = []
                for i in xrange(data_graph.shape[1]):
                    if current[i] != 0:
                        pi_list.append(alpha(past, i, p, q, data_graph)*current[i])
                    else:
                        pi_list.append(0)
                sequence.append(np.random.choice(range(data_graph.shape[1]), p=pi_list/np.linalg.norm(pi_list, ord=1)))
            else:
                sequence.append(sequence[step-1])
    return sequence

## Define word2vec model